## Atomic Swaps
#### 04.5 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2021-11-28

* Transaction groups
* Atomic swap

### Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [ ]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo = cred['MyAlgo']
Alice  = cred['Alice']
Bob    = cred['Bob']
Charlie = cred['Charlie']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error

import json
import pandas as pd
import base64

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

## Atomic Swap

* One of the most import and common operations on a blockchain
* Traditional finance: need trusted intermediary to exchange assets
* On the Algorand blockchain, this type of trade is implemented within the protocol as an Atomic Transfer. 
* Idea: transactions that are part of the transfer either all succeed or all fail.
* Transactions can contain ALOGs or ASA
* Transactions with more than 2 parties are also possible

### A concrete Atomic Swap
"Alice buys 2.5 WSC from Bob for 3.75 Algos"
* Transaction 1: Alice sends Bob 3.75 Algos
* Transaction 2: Bob sends Alice 2.5 WSC

#### Step 0: Get the status before the swap

In [ ]:
# Get the holdings of Alice and Bob separately
alice_holding=asset_holdings_df(algod_client, Alice['public'])
bob_holding=asset_holdings_df(algod_client, Bob['public'])
# Merge in one data.frame using pandas merge
pd.merge(alice_holding, bob_holding,  how="outer", on=["asset-id", "unit", "name", "decimals"], suffixes=['Alice','Bob'])

In [ ]:
# Store the correct ID for the WSC coin
WSC_id=66709453              # <---------- Update!!

#### Step 1a: Prepare transaction 1
This is a simple ALGO payment. Alice sends Bob 3.75 ALGOs

In [ ]:
sp = algod_client.suggested_params()
amt_1 = int( 3.75*1E6 )                     # microalgos!!! 

txn_1 = PaymentTxn(Alice["public"], sp, Bob["public"],amt_1)

#### Step 1b: Prepare transaction 2
This is an ASA transfer. Bob transfers 2.5 WSC coungs to Alice.<br>
Alice has to opt into the WSC coin (In our case, she did so in notebook 04.1!)

In [ ]:
amt_2 = int(2.5 * 1E2)                      # WSC coin is 1/100 divisible !!

txn_2 = AssetTransferTxn(Bob["public"], sp, Alice["public"], amt_2, WSC_id)

#### Step 2: create a TX group

In [ ]:
# group_id calculated from transactions
gid = transaction.calculate_group_id([txn_1, txn_2])

# add group_id to each transactions
txn_1.group = gid
txn_2.group = gid

# This is the gid
print( base64.b32encode(gid).decode() )

#### Step 3: Sign
Everyone has to sign his/her transaction

In [ ]:
# sign transactions
stxn_1 = txn_1.sign(Alice["private"])    
stxn_2 = txn_2.sign(Bob["private"])

#### Step 4: Assemble and submit

In [ ]:
# assemble transaction group
signed_group =  [stxn_1, stxn_2]

txid = algod_client.send_transactions(signed_group)

#### Step 5: Wait for confirmation

In [ ]:
# wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid) 

In [ ]:
algod_client.asset_info(WSC_id)

#### Step 6: Check state after swap

In [ ]:
alice_holding=asset_holdings_df(algod_client, Alice['public'])
bob_holding=asset_holdings_df(algod_client, Bob['public'])
pd.merge(alice_holding, bob_holding,  how="outer", on=["asset-id"], suffixes=['Alice','Bob'])

## Appendix: how to merge dataframes
* The Python library for working with dataframes is called Pandas
* Most people abbreviate it to `pd` using `import pandas as pd`
* The `pd.merge()` command merges two dataframes ... but how exactly?

In [ ]:
alice_holding

In [ ]:
bob_holding

In [ ]:
bob_holding=asset_holdings_df(algod_client, Bob['public'])